In [3]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash' ] 
index = []
data = []

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  1 if c['tag'] == 'BTC' else c['exchanges'][0]['price'], c['algorithm'], c['nethash'] ])
df = pd.DataFrame(data, index=index, columns=columns)
df['block_reward_btc'] = df.block_reward * df.exchange_rate_btc
df['daily_emmisions_btc'] = 86400 / df.block_time * df.block_reward_btc
df

,block_time,block_reward,exchange_rate_btc,algorithm,nethash,block_reward_btc,daily_emmisions_btc
BCH,522.0000,3.125000,3.407627e-03,SHA-256,3199571036374955938,0.010649,1.762566
BTC,570.0000,3.146297,1.000000e+00,SHA-256,860253849123773333333,3.146297,476.912443
DOGE,62.0000,10000.000000,2.583925e-06,Scrypt,3822932887494466,0.025839,36.008239
KAS,1.0007,65.468232,9.121540e-07,kHeavyHash,1343144042084068152,0.000060,5.155949
LTC,149.0000,6.250000,1.186722e-03,Scrypt,2198652621494614,0.007417,4.300870
RVN,60.0000,2500.000000,1.446957e-07,KawPow,6785729430600,0.000362,0.520904


In [4]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
market_data_df = pd.DataFrame(index=index)
market_data_df['hashrate'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / factors_data_df['market_factor']
market_data_df['paying'] = algo_data_df['paying'] * factors_data_df['price_factor'] / sats2btc
market_data_df['daily_consumption_btc'] = market_data_df.hashrate * market_data_df.paying
market_data_df


,hashrate,paying,daily_consumption_btc
SCRYPT,128.242638,0.016481,2.113623
SHA256ASICBOOST,7.211262,0.546415,3.940342
KAWPOW,0.723008,0.087229,0.063067
KHEAVYHASH,54.417478,0.003745,0.203811
